In [1]:
import pandas as pd
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dropout

In [2]:
df = pd.read_csv('train.csv', index_col='id')

In [3]:
df.head()

,title,author,text,label
id,,,,
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isna().sum()

title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df = df.dropna()

In [6]:
X = df.drop(['label'], axis=1)
y = df['label']

In [7]:
X.shape

(18285, 3)

In [8]:
y.shape

(18285,)

In [9]:
##create vocabulary
vocab_size = 6000

## One Hot Representation

In [10]:
books = X.copy()
books['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [11]:
books.reset_index(inplace=True)

In [12]:
import nltk
import re
from nltk.corpus import stopwords

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\subzz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(books)):
    review = re.sub('[^a-zA-Z]', ' ', books['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [15]:
onehot_repr = [one_hot(words, vocab_size) for words in corpus]
onehot_repr

[[1176, 1165, 2387, 4847, 5718, 3283, 187, 2141, 204, 2544],
 [4504, 2681, 4007, 1662, 4887, 4999, 754],
 [1957, 3395, 733, 369],
 [1463, 3933, 2157, 5759, 5031, 2992],
 [624, 4887, 3741, 4595, 4916, 4393, 4887, 5022, 5327, 5437],
 [3700,
  974,
  4270,
  3692,
  834,
  4449,
  324,
  1530,
  5256,
  363,
  2161,
  3483,
  15,
  5884,
  754],
 [3750, 2846, 2289, 2528, 5517, 915, 3487, 5838, 2568, 2005, 3654],
 [2805, 4699, 1820, 93, 621, 2298, 4449, 889, 2568, 2005, 3654],
 [2640, 5156, 3299, 3822, 4418, 5689, 5679, 1922, 4449, 5745],
 [5651, 5654, 2042, 3641, 4540, 208, 878, 2495],
 [4669, 5612, 1262, 2829, 1632, 2652, 1510, 4216, 5883, 1359, 3021],
 [5759, 5051, 5718, 5689, 4449, 621],
 [629, 3098, 2594, 3829, 1327, 659, 2189, 2842, 77],
 [1919, 3145, 4875, 4528, 5674, 4954, 3620, 2568, 2005, 3654],
 [4810, 3792, 900, 1674, 1953, 2568, 2005, 3654],
 [1875, 3785, 5457, 3987, 5805, 1477, 101, 1369, 4518, 818],
 [1543, 2637, 2681],
 [5636, 509, 3565, 4287, 4449, 5084, 5081, 754],
 [3213

## Embedding Representation

In [16]:
sent_len = 30
embedded_doc = pad_sequences(onehot_repr, padding='pre', maxlen=sent_len)
embedded_doc

array([[   0,    0,    0, ..., 2141,  204, 2544],
       [   0,    0,    0, ..., 4887, 4999,  754],
       [   0,    0,    0, ..., 3395,  733,  369],
       ...,
       [   0,    0,    0, ..., 2568, 2005, 3654],
       [   0,    0,    0, ..., 4239, 4676, 2879],
       [   0,    0,    0, ..., 2593, 2937, 5371]])

## Model

In [17]:
embedding_vector_features=40
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length=sent_len))
model.add(Dropout(0.4))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.add(Dropout(0.2))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 40)            240000    
_________________________________________________________________
dropout (Dropout)            (None, 30, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
_________________________________________________________________
dropout_1 (Dropout)          (None, 1)                 0         
Total params: 296,501
Trainable params: 296,501
Non-trainable params: 0
_________________________________________________________________


In [18]:
import numpy as np
X_new = np.array(embedded_doc)
y_new = np.array(y)

In [19]:
X_new.shape

(18285, 30)

In [20]:
y_new.shape

(18285,)

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.30, random_state=42)

## Model Training

In [23]:
model.fit(X_train,y_train,epochs=10,batch_size=70, validation_data=(X_test,y_test))

Epoch 1/10
183/183 [==============================] - 8s 43ms/step - loss: 1.6411 - accuracy: 0.7744 - val_loss: 0.2379 - val_accuracy: 0.9092
Epoch 2/10
183/183 [==============================] - 8s 45ms/step - loss: 1.5130 - accuracy: 0.8548 - val_loss: 0.3214 - val_accuracy: 0.8489
Epoch 3/10
183/183 [==============================] - 9s 50ms/step - loss: 1.4981 - accuracy: 0.8522 - val_loss: 0.2891 - val_accuracy: 0.8791
Epoch 4/10
183/183 [==============================] - 10s 53ms/step - loss: 1.4112 - accuracy: 0.8630 - val_loss: 0.2453 - val_accuracy: 0.9056
Epoch 5/10
183/183 [==============================] - 9s 50ms/step - loss: 1.4076 - accuracy: 0.8701 - val_loss: 0.2483 - val_accuracy: 0.9019
Epoch 6/10
183/183 [==============================] - 9s 47ms/step - loss: 1.4079 - accuracy: 0.8819 - val_loss: 0.2247 - val_accuracy: 0.9140
Epoch 7/10
183/183 [==============================] - 8s 44ms/step - loss: 1.4273 - accuracy: 0.8848 - val_loss: 0.2347 - val_accuracy: 0.909

## Model Performance and Accuracy

In [30]:
y_pred = model.predict_classes(X_test)

In [27]:
from sklearn.metrics import confusion_matrix, classification_report

In [31]:
print(confusion_matrix(y_pred,y_test))
print('\n')
print(classification_report(y_pred,y_test))

[[2821  172]
 [ 286 2207]]


              precision    recall  f1-score   support

           0       0.91      0.94      0.92      2993
           1       0.93      0.89      0.91      2493

    accuracy                           0.92      5486
   macro avg       0.92      0.91      0.92      5486
weighted avg       0.92      0.92      0.92      5486

